Task1_0620. 네이버 영화 순위 사이트에서 영화제목, 가격, 타입(구매/대여) 정보를 가져와서
Title, Price, Type 3개의 컬럼으로 구성된 100개 데이터포인트로 구성된 데이터 프레임을 출력하세요.

In [22]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.request as rq

url = "https://serieson.naver.com/v3/movie/ranking/realtime"

html = rq.urlopen(url)
bs = BeautifulSoup(html, 'html.parser')

titles = [i.text for i in bs.select('span.Title_title__s9o0D')]

types = [i.text for i in bs.select('span.Price_text__pRk_f')]

prices = [re.sub('캐시', '', i.text) for i in bs.select('span.Price_price__GqXqo')]

df = pd.DataFrame({
  'Title':titles,
  'Type':types,
  'Price':prices
})

df

,Title,Type,Price
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),구매,"7,150"
1,콩나물,구매,"1,000"
2,태극기 휘날리며 - 특별판,구매,"10,000"
3,설계자,대여,"11,000"
4,원피스 필름 레드,대여,"1,540"
...,...,...,...
81,마스크,대여,"1,300"
82,사운드 오브 뮤직,구매,"5,500"
83,마지막 한 걸음까지,구매,500
84,흑인 오르페,구매,500


Task2_0620. 앞에서 출력한 기사 리스트를 pandas 데이터프레임으로 변환 후 csv파일로 저장 후 다시불러오세요. (컬럼은 인덱스와 텍스트 2개 컬럼으로 구성)

In [25]:
import requests as rq
import pandas as pd

# 다음 정치 카테고리 뉴스 크롤링

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}

url = 'https://news.daum.net/politics#1'

r = rq.get(url, headers=headers)
html = r.text
soup = BeautifulSoup(html, 'html.parser')

lines = soup.select('body > div > main > section > div > div > ul > li > div > div > strong > a')

titles = [line.text for line in lines]
df = pd.DataFrame({
  'Index':range(1,len(titles)+1),
  'Title':titles
})
df.to_csv('news_titles.csv', index=None)

In [26]:
df = pd.read_csv('news_titles.csv')

df.head()

,Index,Title
0,1,"이준석, 홍준표의 ‘이재명=동탁’ 비유에 “판단 잘못해 일 그르친 원소로 보여”"
1,2,한미동맹엔 없지만 북중동맹에 있는 것…새로운 북러 조약에도? [스프]
2,3,'부안 지진' 중대본 1단계 10일 만에 해제… 시설피해 1196건
3,4,[영상] “추미애 탄핵 6행시? 6선 의심스러운 저급함에 민망”
4,5,"신원식, 폴란드 민스크 공군기지 방문…FA-50 전력화 확인"


Task3_0620. url = 'https://news.daum.net/politics#' 정치기사 1페이지~10페이지에 있는 기사를 모두 출력하세요.

In [27]:
import requests as rq
from bs4 import BeautifulSoup

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}

url = 'https://news.daum.net/politics#{}'

for i in range(1,11):
  #res = rq.get(url+str(i), headers=headers)
  res = rq.get(url.format(str(i)), headers=headers)
  soup = BeautifulSoup(res.text, 'html.parser')
  lines = soup.select('a.link_txt')
  titles = [line.text for line in lines]

df = pd.DataFrame({
  'Index':range(1,len(titles)+1),
  'Title':titles
})
df.to_csv('news_titles2.csv', index=None)



In [29]:
df = pd.read_csv('news_titles2.csv')

df.tail()

,Index,Title
33,34,제휴 언론사
34,35,배열이력
35,36,전체뉴스
36,37,연재
37,38,팩트체크


In [37]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}
url = 'https://finance.naver.com/news/mainnews.naver?date=2024-06-20&page={}'

df = pd.DataFrame()
for page in range(1,7):
  res = rq.get(url.format(page), headers=headers)
  soup = BeautifulSoup(res.text, 'html.parser')

  lines = soup.select('dd.articleSubject > a')
  titles = [line.text for line in lines]
  df[str(page)+'Page'] = titles

#df.to_csv('news_titles3.csv', index=None)
df


,1Page,2Page,3Page,4Page,5Page,6Page
0,"뉴욕증시, AI 열풍에 S&P500 5천500선 첫 돌파하며 상승 출발","""내년엔 세금 22%, 수익낸 펀드 일단 팔자""…장기투자 막는 금투세","깐깐한 상장심사… ""투자자 보호"" vs ""성장기업 위축""[신하연의 여의도 돋보기]",“배당도 먹고 차익도 얻자”…밸류업 바람에 ‘이 종목’ 더 눈길가네,"SK, 사업 리밸런싱 '시동'…SK이노 ""합병 등 전략적 방안 검토""(종합)","“워런 버핏이 코스피에 투자했다면?”…2800선 넘어선 코스피, ‘삼천피’까지 달릴..."
1,“이대로가면 시장 다 죽어”…금투세 폭탄에 ‘펀드런 공포’ 확산,'이사의 주주충실의무' 너무나 상식적…배임죄 폐지 없이도 도입 가능,“주식 좀 한다는 사람은 다 가지고있다”...150조 돌파한 ‘이것’ 쟁탈전,"엔비디아, 또 호재 터졌다…""美 큰손 '15조원'어치 매수 예정""","2차전지 3총사, 투자유치로 반등 노린다","네이버웹툰 나스닥 상장, 네이버 '게임체인저' 낙관 어렵다? [투자360]"
2,엔비디아의 힘…S&P 5500 `첫 경험`,"""시장 좋을때 현금 확보""… 사옥 파는 기업들","""잘 나가면 뭐하나""...투자처 부족한 인도증시","미래에셋증권, 업계 최초 해외주식 30조원·연금 30조원 달성",“코스닥 2분기 영업익 ‘반토막’”,"SOL 미국 AI반도체 칩메이커 ETF, 1개월 수익률 19.59%"
3,엔비디아 또 나홀로 질주… 시총 1위 `넘사벽`?,"""中 부동산 내년부터 반등…亞 증시 여전히 기회 많아""[단독인터뷰]","SK이노베이션, SK E&S와 합병 가능성에 15% 뛰었다[핫종목]","IMM-맥쿼리, 이르면 내주 제뉴원사이언스 지분 100% SPA 체결","코스피, 2800선 아슬아슬…저출산 테마주 급등",KB 이어 한화도 7월 브랜드명 바꾼다… 올 여름 'ETF' 경쟁 후끈
4,"뉴욕증시, 혼조세…'엔비디아 랠리'에 S&P500 장중 최고치",밸류업 확정되자…팔기 바쁜 연기금,"'반도체 랠리' 타고 2800 벽 넘은 코스피…""3100까지"" vs ""과열 주의""","'국내 M7 ETF', 개인이 한달 내내 사들였다",미래에셋證 해외주식 '30조' 돌파…올해만 6.7조 늘어,신한금융그룹의 공개매수 재테크… 제이시스메디칼에 500억 베팅
5,“자네에게 기대가 크네”…1년만에 영업익 10배 오른다는 ‘이 종목’ 무엇?,ETF 시장 급성장에…'EMP 펀드' 뜬다,SK이노 `웃고` SK `울고`… SK株 희비,"밸류업 성공, 미국보다 유럽?···코스피 외인자본 중 절반 영국","코스피 6% 올랐는데 빚투 개미들은 ‘곡소리’... 반대매매 급증, 왜?","SK이노 18%↑, SK(주) 5%↓…합병설에 엇갈린 SK그룹株"
6,"“올해 번 돈, 다 날아갔네”…선거 앞두고 고꾸라진 ‘이 나라’ 증시",역대급 '바이 코리아' 코스피 2800선 탈환,"외인 '이 업종' 쓸어담더니…""코스피 또 신고가"" 이대로 3000까지?",증권성 의혹 벗은 이더리움 2% 반등 [매일코인],"[단독] 에코프로비엠, 최대 3조원 규모로 투자 유치 추진… FI 물밑 접촉",2차전지株 3총사 기지개…투자유치로 반등 노린다 [투자360]
7,“외국인 웰컴”…역대급 ‘바이 코리아’에 코스피 2800선 탈환,'반도체 봄' 여름까지…삼전·하이닉스 방긋,"""마취 후 못 깨면 어쩌나""...반려견도 캡슐내시경으로 건강 챙긴다",[단독] 위기의 위워크 입주 해외투자 건물···대출 연장 첫 사례 나왔다,엔비디아 주가는 ‘고공행진’인데…내가 투자한 AI주는 왜?,"'SK E&S 합병설' SK이노베이션, 주가 19%↑[ET라씨로]"
8,"""국민연금만으로는 부족해""…은퇴 후 '평생 돈줄' 전략은?[555 기획]",비트코인·이더리움 `디커플링`… 한 달새 엇갈린 수익률,코스피 2년5개월만에 2800선 안착…하반기 3000 넘길까,통신비 연체자 37만명...최대 90% 감면받는다,"""K-방산 잘나가네""…방산株 상승세 지속될까",“10대 수출국인데 23년째 신흥국”…내일 韓증시 선진국편입 무산 확실시 [투자360]
9,눌러앉는 회계사들…빅4 회계법인 채용문 더 좁아진다[마켓인],"""中 부동산 내년부터 반등…내수기업 투자 유망""[단독인터뷰]","“코스닥을 왜 샀을까""... 올해 마이너스 수익률에 투자자 ‘울상’",`비트코인 6만5000달러 또 붕괴…엔비디아로 `투자이민`?,"또 터진 불량 공모주 사태…이노그리드, 사상 첫 상장승인 취소","""루마니아 K9 자주포 수출""…""아직 확정 아니야"""


In [32]:
df = pd.read_csv('news_titles3.csv')

df

,Index,Title
0,1,"“워런 버핏이 코스피에 투자했다면?”…2800선 넘어선 코스피, ‘삼천피’까지 달릴..."
1,2,"네이버웹툰 나스닥 상장, 네이버 '게임체인저' 낙관 어렵다? [투자360]"
2,3,"SOL 미국 AI반도체 칩메이커 ETF, 1개월 수익률 19.59%"
3,4,KB 이어 한화도 7월 브랜드명 바꾼다… 올 여름 'ETF' 경쟁 후끈
4,5,신한금융그룹의 공개매수 재테크… 제이시스메디칼에 500억 베팅
5,6,"SK이노 18%↑, SK(주) 5%↓…합병설에 엇갈린 SK그룹株"
6,7,2차전지株 3총사 기지개…투자유치로 반등 노린다 [투자360]
7,8,"'SK E&S 합병설' SK이노베이션, 주가 19%↑[ET라씨로]"
8,9,“10대 수출국인데 23년째 신흥국”…내일 韓증시 선진국편입 무산 확실시 [투자360]
9,10,"""루마니아 K9 자주포 수출""…""아직 확정 아니야"""
